In [1]:
import os
from glob import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets , transforms

import numpy as np

In [4]:
seed = 1  # shuffle 을 하더라도, seed 를 줘서 같은 결과가 나오도록 함 
batch_size = 64
test_batch_size = 64

no_cuda = False

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else "cpu")

## Preprocess

In [7]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train = True , download = True,
                      transform = transforms.Compose([  # argumentation 툴 - 노이즈를 줄수있는함수 등 다양한것을 한번에 처리해줌 
                          transforms.ToTensor(),
                          transforms.Normalize((0.1307,), (0.3081,))  # 정규화 - 여기서 전처리까지 같이 해서 나옴 
                      ])),
    batch_size = batch_size, shuffle = True )


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train = False , transform= transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size = test_batch_size, shuffle = True )

## Model

In [31]:
class Net(nn.Module):  # 이 클래스 안에 , 역전파까지 가능하게 들어가있음. 그래서 우린 어떤 모델 포워딩 할지만 정하면 됨 
    def __init__(self):
        super(Net, self).__init__()  # 사용할 히든 스테이트를 정의
        self.conv1 = nn.Conv2d(1, 20 , 5 , 1)
        self.conv2 = nn.Conv2d(20, 50 , 5, 1)
        self.fc1 = nn.Linear( 4 *4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x ):  # 그 hidden state 가 어떻게 흘러가는지를 보여줌 
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim  =1 )

## Optimization
- Model 과 Optimization 설정

**확률적 경사 하강법 ( SGD , Stochastic Gradient Descent )**

: 매개변수 값을 조정시 ( 매 스텝 ( step ) 에서 ), 전체 데이터가 아니라 랜덤으로 선택한 딱 1개의 샘플에 대해서 gradient를 계산한다.

**모멘텀 ( Momentum )** 

: 관성이라는 물리학 법칙 처럼, 이동 벡터를 이용해 이전 **기울기**의 영향을 받도록 하는 것.

: **이전 벡터의 이동의 크기를 현재에 반영해주는 것**이다.

[관련해서 정리한 내용](https://hazel01.tistory.com/36?category=897675)

In [32]:
model = Net().to(device) # 모델을 불러옴
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.5 ) # 오티마이저를 불러옴

- Parameters 들 확인
    - Weight, Bais 를 순서대로 보여줌 

In [33]:
params = list(model.parameters()) # 파라미터를 볼수 있음
for i in range(8):
    print(params[i].size())

torch.Size([20, 1, 5, 5])
torch.Size([20])
torch.Size([50, 20, 5, 5])
torch.Size([50])
torch.Size([500, 800])
torch.Size([500])
torch.Size([10, 500])
torch.Size([10])


# Befor Training

- 학습하기 전에 Model이  Train 할 수 있도록  Train mode로 변환
    - Convolution 또는 Linear 뿐만 아니라, dropout, batch normalization 과 같이 파라미터를 가진 레이어들도 학습하기 위해 준비 

In [34]:
model.train() # trian mode 

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- 모델에 넣기 위한 첫 Batch 데이터 추출

In [35]:
data, target = next(iter(train_loader))
data.shape, target.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- 추출한 batch 데이터를 cpu 또는 gpu 와 같은 device에 compile 
     - 겉으로는 차이가 없지만, gpu를 할당받기 위해서? 여튼 compile 해준다.

In [36]:
data, target = data.to(device), target.to(device)
data.shape, target.shape 

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

- gradients를 clear해서 새로운 최적화 값을 찾기 위해 준비

In [37]:
optimizer.zero_grad()  # zero_grad를  이용하면, clear 해짐

- 준비한 데이터를 model 에 input 으로 넣어 output을 얻음

In [38]:
output = model(data) # 이후, model에 넣어줌 keras는 보통 perdict , torch는 output이라 보통 씀

- Model 에서 예측한 결과를 Loss Functoon 에 넣음
    - 여기서는 Negative Log-likelihood loss 라는 loss function을 사용

In [39]:
loss = F.nll_loss(output, target)

- Back Propagation을 통해 Gradients를 계산

In [40]:
loss.backward()

- 계산된 Gradients는 계산한 것으로 마무리 되는 것이 아니라 , Parameter 에 Update

In [41]:
optimizer.step() # update를 해주는 것. 계산된 결과를 넣어주는 것 

- 여기까지가 원스텝! 

# Evaluation
- 앞에서 model.train() 모드로 변한 것처럼 평가할 때는 model.eval()로 설정
    - batch normaliztion 이나 drop out 같은 layer 들을 잠금


In [47]:
model.eval() 

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

- autograd engine, 즉 backpropagation 이나 gradient 계산 등을 껴서 memory usage 를 줄이고 속도를 높임 

In [48]:
test_loss = 0
correct  = 0

with torch.no_grad():
    data, target = next(iter(test_loader))
    data, target = data.to(device), target.to(device)
    output = model(data)
    
    # 계산용으로 loss 를 쌓음
    test_loss += F.nll_loss(output, target, reduction= 'sum' ).item() #reduction 이걸 안하면, 배치사이즈를 기준으로 따로 계산해주는데, 이걸 sum 해주면 전체 데이터에 대해합쳐서 넣어줌
    
    pred = output.argmax(dim = 1 , keepdim = True ) # 차원수는 계속 유지
    correct = pred.eq(target.view_as(pred)).sum() # True, False로 나온 값을 sum 함  - 같으게 얼마나 많은지 check 하는것 
    
 

In [49]:
test_loss

22.667163848876953

In [53]:
print(output.shape)
print(pred.shape)
print(target.view_as(pred).shape)

torch.Size([64, 10])
torch.Size([64, 1])
torch.Size([64, 1])


In [55]:
# 한번 넣었을때(배치사이즈 ) 얼마나 맞췄는지 확인
pred.eq(target.view_as(pred)).sum().item() / 64 # 총 이미지 갯수가 64 개 

0.90625

In [58]:
test_loss /= len(test_loader.dataset)
test_loss

2.2667163848876954e-11

# Start Training 

In [65]:
epochs = 2
log_interval = 100  # 몇번 로그만에 보여줄지 

for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()
    for batch_idx,  (data, target ) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # 데이터를 뽑아줌 
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step() 
        # -------- 여기까지가 한 스텝임  ------- #
         
        # 잘 진행되고 있는지 확인하기 위한코드
        if  batch_idx % log_interval == 0 :
            print("Train Epoch : {} [{}/{} ({:.0f}%)]   Loss : {:.6f}".format(
            epoch, batch_idx * len(data), len(train_loader.dataset),  # 전체 데이터 셋
            100* batch_idx / len(train_loader), loss.item()
            ))
            
            
    model.eval() # 한 에폭에 따른 평가
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction = 'sum').item()
            pred = output.argmax(dim  =1 , keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set : Average Loss : {:.4f}, Accuract : {}/{} ({:.0f}%)'.format(
            test_loss, correct, len(test_loader.dataset), 100* correct / len(test_loader.dataset)))

Train Epoch : 1 [0/60000 (0%)]   Loss : 0.235288
Train Epoch : 1 [6400/60000 (11%)]   Loss : 0.203309
Train Epoch : 1 [12800/60000 (21%)]   Loss : 0.178280
Train Epoch : 1 [19200/60000 (32%)]   Loss : 0.242543
Train Epoch : 1 [25600/60000 (43%)]   Loss : 0.264136
Train Epoch : 1 [32000/60000 (53%)]   Loss : 0.170799
Train Epoch : 1 [38400/60000 (64%)]   Loss : 0.277752
Train Epoch : 1 [44800/60000 (75%)]   Loss : 0.113177
Train Epoch : 1 [51200/60000 (85%)]   Loss : 0.139957
Train Epoch : 1 [57600/60000 (96%)]   Loss : 0.276587
Test set : Average Loss : 0.1952, Accuract : 9450/10000 (94%)
Train Epoch : 2 [0/60000 (0%)]   Loss : 0.140250
Train Epoch : 2 [6400/60000 (11%)]   Loss : 0.051344
Train Epoch : 2 [12800/60000 (21%)]   Loss : 0.204236
Train Epoch : 2 [19200/60000 (32%)]   Loss : 0.082684
Train Epoch : 2 [25600/60000 (43%)]   Loss : 0.120147
Train Epoch : 2 [32000/60000 (53%)]   Loss : 0.223981
Train Epoch : 2 [38400/60000 (64%)]   Loss : 0.080957
Train Epoch : 2 [44800/60000 (75